$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

In [1]:
import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [2]:
# Parameters
N = 1  # Number of sites
cutoff = 16    # Basis cutoff
a = 1.0       # Lattice spacing

In [3]:
def tensor_identity(size, site, total_sites, operator):

    I = np.eye(size, dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
    
    return reduce(np.kron, operators) 

In [4]:
# Initialize the Hamiltonian
I_b = np.eye(cutoff ** N)
I_f = np.eye(2 ** N)
H = np.zeros(np.kron(I_b,I_f).shape, dtype=np.complex128)

# Define operators
q = create_matrix(cutoff, 'q')
p = create_matrix(cutoff, 'p')
chi = np.array([[0, 1], [0, 0]])
chidag = np.array([[0, 0], [1, 0]])


for n in range(N):

    q_n = tensor_identity(cutoff, n, N, q)
    q_np1 = tensor_identity(cutoff, (n + 1) % N, N, q)
    q_nm1 = tensor_identity(cutoff, (n - 1) % N, N, q)

    p_n = tensor_identity(cutoff, n, N, p)

    chi_n = tensor_identity(2, n, N, chi)
    chidag_n = tensor_identity(2, n, N, chidag)
    chi_np1 = tensor_identity(2, (n + 1) % N, N, chi)
    chidag_np1 = tensor_identity(2, (n - 1) % N, N, chidag)
    
    # Kinetic term
    p2 = np.matmul(p_n,p_n) / (2 * a)
    kinetic_term = np.kron(p2, I_f)
    
    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b #W''(q) = 1
    potential = np.matmul(W_prime,W_prime)*(a / 2)
    potential_term = np.kron(potential, I_f)

    # Fermionic term
    commutator_term = (-1) ** n * np.kron(W_double_prime, (np.matmul(chidag_n, chi_n) - 0.5 * I_f))

    H += (kinetic_term+potential_term+commutator_term)

    # Cross site terms
    if 0 < n < N - 1:    
        gradient = (q_np1 - q_nm1) / (2 * a)
        gradient_term = np.kron((a / 2) * np.matmul(gradient,gradient), I_f)
        potential_gradient_term = np.kron(a * np.matmul(W_prime,gradient), I_f)

        # Fermionic hopping
        fermion_hopping = (np.matmul(chidag_n,chi_np1) + np.matmul(chidag_np1,chi_n)) / (2 * a)
        fermion_hopping_term = np.kron(I_b, fermion_hopping)

        H += (potential_gradient_term + fermion_hopping_term) 
   

In [5]:
np.sort(np.linalg.eig(H)[0].real)[:7]

array([-1.11022302e-16,  1.00000000e+00,  1.00000000e+00,  2.00000000e+00,
        2.00000000e+00,  3.00000000e+00,  3.00000000e+00])

In [6]:
print(kinetic_term.size,
potential_term.size,
commutator_term.size)

1024 1024 1024


# Sparse matrix form

$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

In [7]:
import scipy
from scipy.sparse import eye, kron, coo_matrix
from scipy.sparse.linalg import eigsh

In [158]:
# Parameters
N = 2
cutoff = 16
a = 1.0

In [159]:
def tensor_identity(size, site, total_sites, operator, boundary_type='dirichlet'):

    I = eye(size, format='coo', dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
  
    return reduce(kron, operators) 

In [164]:
# Initialize the Hamiltonian as a sparse matrix
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')
dim = I_b.size * I_f.size
H = coo_matrix((dim, dim), dtype=np.complex128)

# Define operators as sparse matrices
q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

for n in range(N):
    
    q_n = tensor_identity(cutoff, n, N, q)
    p_n = tensor_identity(cutoff, n, N, p)

    chi_n = tensor_identity(2, n, N, chi)
    chidag_n = tensor_identity(2, n, N, chidag)
    

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  
    kinetic_term = kron(p2, I_f, format='coo') 

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = coo_matrix(W_prime @ W_prime * (a / 2))
    potential_term = kron(potential, I_f, format='coo')

    # Fermionic term
    commutator_term = (-1) ** n * kron(W_double_prime, coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')

    H += kinetic_term + potential_term + commutator_term 

    # Cross-site terms
    # Set operators to zero matrices at boundary
    zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
    zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

    # Dirichlet BC's
    if n < N-1:
        q_np1 = tensor_identity(cutoff, (n + 1), N, q)
        chi_np1 = tensor_identity(2, (n + 1), N, chi)
        chidag_np1 = tensor_identity(2, (n - 1) , N, chidag)
    else:
        q_np1 = zero_qop
        chi_np1 = zero_cop
        chidag_np1 = zero_cop

    if n == 0:
        q_nm1 = zero_qop
    else:
        q_nm1 = tensor_identity(cutoff, (n - 1), N, q)

    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = kron((a / 2) * (gradient @ gradient), I_f, format='coo')
    potential_gradient_term = kron(a * (W_prime @ gradient), I_f, format='coo')

    # Fermionic hopping
    fermion = coo_matrix((chidag_n @ chi_np1 + chidag_np1 @ chi_n)) / (2 * a)
    fermion_term = kron(I_b, fermion, format='coo')

    H += potential_gradient_term + fermion_term 


In [165]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 4
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [-0.11803399  0.88196601  0.88196601  1.        ]


In [139]:
np.sort(scipy.sparse.linalg.eigs(H,k=6,sigma=0.0)[0].real)

array([0.06985204, 0.62044593, 1.06985204, 1.62044593, 3.387683  ,
       3.39135356])